In [1]:
from edc import print_info 
print_info("geodb-manage")


***Notebook Title***  
GeoDB: Manage Datasets

***Notebook Description***  
Euro Data Cube GeoDB: Manage Datasets


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB


In [2]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `GEODB_API_SERVER_PORT`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_AUTH_DOMAIN`
* `SH_CLIENT_ID`, `SH_CLIENT_NAME`, `SH_CLIENT_SECRET`, `SH_INSTANCE_ID`
* `XCUBE_GEN_API_SERVER_PORT`, `XCUBE_GEN_API_SERVER_URL`, `XCUBE_GEN_API_USER_NAME`, `XCUBE_GEN_AUTH_AUD`, `XCUBE_GEN_AUTH_CLIENT_ID`, `XCUBE_GEN_AUTH_CLIENT_SECRET`, `XCUBE_GEN_AUTH_DOMAIN`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_ACCESS_KEY_ID`, `AWS_BUCKET`, `AWS_SECRET_ACCESS_KEY`
* `DAPA_URL`


In [3]:
from edc import check_compatibility
check_compatibility("user-0.19.6")

This notebook is compatible with this base image version (user-0.19.6).

## Manage Collections in your GeoDB



### Connecting to the GeoDB

In [6]:
from xcube_geodb.core.geodb import GeoDBClient

In [7]:
geodb = GeoDBClient()

In [8]:
# If you are logged in, this will tell you what account the system currently uses
geodb.whoami

'geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd'

In [9]:
# Lets get already existing collections
ds = geodb.get_my_collections()
ds

,table_name


### Creating collections

Once the connection has been established you will be able to create a collection. The collection will contain standard properties (fields) plus custom properties
which you can add at your disgretion. Please use [PostGreSQL type definitions](https://www.postgresql.org/docs/11/datatype.html). We recommend stying simple with
your data types as we have not tested every single type.

In [30]:
# geodb.drop_collection('land_use')

# Have a look at fiona feature schema
collections = {
        "land_use": 
        {
            "crs": 3794,
            "properties": 
            {
                "RABA_PID": "float", 
                "RABA_ID": "float", 
                "D_OD": "date"
            }
        }
    }


geodb.create_collections(collections)

{'collections': {'geodb_admin_land_use': {'crs': 3794,
                                          'properties': {'D_OD': 'date',
                                                         'RABA_ID': 'float',
                                                         'RABA_PID': 'float'}}}}

In [31]:
ds = geodb.get_my_collections()
ds

,table_name
0,land_use
1,test
2,test2
3,geodb_admin_land_use


### Loading data into a dataset

Once the table has been created, you can load data into the dataset. The example below loads a shapefile. The attributes of the shapefile correspond to the dataset's properties.


In [7]:
import geopandas
gdf = geopandas.read_file('data/sample/land_use.shp')
gdf

,RABA_PID,RABA_ID,D_OD,geometry
0,4770326.0,1410,2019-03-26,"POLYGON ((453952.629 91124.177, 453952.696 911..."
1,4770325.0,1300,2019-03-26,"POLYGON ((453810.376 91150.199, 453812.552 911..."
2,2305689.0,7000,2019-02-25,"POLYGON ((456099.635 97696.070, 456112.810 976..."
3,2305596.0,1100,2019-02-25,"POLYGON ((455929.405 97963.785, 455933.284 979..."
4,2310160.0,1100,2019-03-11,"POLYGON ((461561.512 96119.256, 461632.114 960..."
...,...,...,...,...
9822,6253989.0,1600,2019-03-08,"POLYGON ((460637.334 96865.891, 460647.927 969..."
9823,6252044.0,1600,2019-03-26,"POLYGON ((459467.868 96839.686, 459467.770 968..."
9824,6245985.0,2000,2019-04-08,"POLYGON ((459488.998 94066.248, 459498.145 940..."
9825,6245986.0,2000,2019-02-20,"POLYGON ((459676.680 94000.000, 459672.469 939..."


In [8]:
geodb.insert_into_collection('land_use', gdf)

Data inserted into land_use

In [9]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od,testä_prop
0,19657,2020-05-08T06:51:17.034744+00:00,None,"POLYGON ((456099.635 97696.070, 456112.810 976...",2305689,7000,2019-02-25,None
1,19680,2020-05-08T06:51:17.034744+00:00,None,"POLYGON ((459898.930 100306.841, 459906.288 10...",2301992,7000,2019-04-06,None
2,19749,2020-05-08T06:51:17.034744+00:00,None,"POLYGON ((459591.248 92619.056, 459592.745 926...",2333229,7000,2019-02-20,None
3,19769,2020-05-08T06:51:17.034744+00:00,None,"POLYGON ((459013.303 100354.458, 459022.756 10...",2336738,7000,2019-03-15,None
4,19783,2020-05-08T06:51:17.034744+00:00,None,"POLYGON ((460851.200 93442.039, 460846.405 934...",6292562,7000,2019-02-21,None
...,...,...,...,...,...,...,...,...
763,38900,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((456523.670 94000.000, 456550.368 939...",6187993,7000,2019-03-13,None
764,39027,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((460793.279 94000.000, 460797.466 939...",6219403,7000,2019-02-21,None
765,39041,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((461750.000 98434.292, 461750.000 984...",6219602,7000,2019-03-20,None
766,39105,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((463628.912 97501.218, 463636.859 975...",6215237,7000,2019-03-20,None


### Delete from a Collection

In [10]:
geodb.delete_from_collection('land_use', query="raba_id=eq.7000")

Data from land_use deleted

In [11]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,Empty Result


### Updating a Collection

In [12]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od,testä_prop
0,19656,2020-05-08T06:51:17.034744+00:00,None,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2019-03-26,None
1,10,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2000-01-01,None
2,19664,2020-05-08T06:51:17.034744+00:00,None,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2019-03-14,None
3,2,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2000-01-01,None
4,5944,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((463488.069 90719.797, 463489.977 907...",5955535,1300,2000-01-01,None
...,...,...,...,...,...,...,...,...
3575,39247,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2019-03-04,None
3576,39248,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2019-04-01,None
3577,39279,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2019-01-05,None
3578,39280,2020-05-11T11:18:16.044507+00:00,None,"POLYGON ((461378.845 91124.318, 461490.224 911...",6264865,1300,2019-02-21,None


In [13]:
geodb.update_collection('land_use', query="raba_id=eq.1300", values={'d_od': '2000-01-01'})

land_use updated

In [14]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od,testä_prop
0,5944,2020-01-31T11:08:35.241344+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((463488.069 90719.797, 463489.977 907...",5955535,1300,2000-01-01,None
1,19740,2020-05-08T06:51:17.034744+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((454709.766 97354.278, 454704.878 973...",2331038,1300,2000-01-01,None
2,19741,2020-05-08T06:51:17.034744+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((453820.737 98574.017, 453816.740 985...",2357574,1300,2000-01-01,None
3,19767,2020-05-08T06:51:17.034744+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((463350.273 100002.359, 463353.150 10...",2335009,1300,2000-01-01,None
4,1763,2020-01-31T11:08:35.241344+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((458678.881 99685.980, 458678.361 996...",210072,1300,2000-01-01,None
...,...,...,...,...,...,...,...,...
3575,39246,2020-05-11T11:18:16.044507+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((458304.555 100146.103, 458306.726 10...",6268058,1300,2000-01-01,None
3576,39247,2020-05-11T11:18:16.044507+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2000-01-01,None
3577,39248,2020-05-11T11:18:16.044507+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2000-01-01,None
3578,39279,2020-05-11T11:18:16.044507+00:00,2020-05-11T11:18:26.342697+00:00,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2000-01-01,None


### Managing Properties of a Collection

In [15]:
geodb.get_my_collections()

,table_name
0,land_use
1,test
2,test2
3,geodb_admin_land_use


In [16]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,testä_prop,integer


In [17]:
geodb.add_property('land_use', "test_prop", 'integer')

Properties added

In [18]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,testä_prop,integer


In [19]:
geodb.drop_property('land_use', 'test_prop')

Properties ['test_prop'] dropped from geodb_admin_land_use

In [20]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,testä_prop,integer


In [21]:
geodb.add_properties('land_use', properties={'test1': 'integer', 'test2': 'date'})

Properties added

In [22]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,testä_prop,integer


In [23]:
geodb.drop_properties('land_use', properties=['test1', 'test2'])

Properties ['test1', 'test2'] dropped from geodb_admin_land_use

In [8]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,geodb_admin_land_use,id,integer
1,geodb_admin_land_use,created_at,timestamp with time zone
2,geodb_admin_land_use,modified_at,timestamp with time zone
3,geodb_admin_land_use,geometry,USER-DEFINED
4,geodb_admin_land_use,raba_pid,double precision
5,geodb_admin_land_use,raba_id,double precision
6,geodb_admin_land_use,d_od,date
7,geodb_admin_land_use,testä_prop,integer


In [29]:
geodb.drop_collection('land_use')

Collection ['geodb_admin_land_use'] deleted